# <span style="color:#72A98F">Location from Timestamp Notebook</span>

Hello Person! In this notebook I will be using the *[LLAT](../../Data/Location/LIGHT-1_LLA_April.csv)* (Latitude Longitude Altitude Time) file to find the location at a given **Event Timestamp**.

## <span style="color:#72A98F">Importing libraries and giving constant values</span>

In [4]:
import numpy as np
import pandas as pd
from astropy.time import Time
import raadpy as rp

LocationFile = '../../Data/Location/LIGHT-1_LLA_April.csv'
LLATData = pd.read_csv(LocationFile)

## <span style="color:#72A98F">Finding Timestamp location for a specific value</span>

In [50]:
# GivenTimestamp = E.get_mjd()
# A remake of this line above will be used to actually get the timestamp ^
# But for now, we will name a value that exists in the range of april.
GivenTimestamp = 59671

TData = LLATData["Time (ModJDate)"]

#Count the rows and look through them.
for row in range(len(TData)):

    #
    if GivenTimestamp == TData[row]:
        MaximumData = LLATData.T[row]
        MinimumData = LLATData.T[row]
        break
    
    if TData[row] >= GivenTimestamp:
        MaximumData = LLATData.T[row]
        MinimumData = LLATData.T[row-1]
        break

TimeRange = [MinimumData[0], MaximumData[0]]
LongRange = [MinimumData[2], MaximumData[2]]
LatRange  = [MinimumData[1], MaximumData[1]]
longitude2 = np.interp(GivenTimestamp, TimeRange, LongRange)
latitude2 = np.interp(GivenTimestamp, TimeRange, LatRange)

TS = [] #For the values to be added together.
TS.append([GivenTimestamp, longitude2, latitude2])
TS

[[59671, 46.938, -48.851]]

## <span style="color:#72A98F">Finding Timestamp Locations for an Array of Timestamps

In [46]:
TSA  = [59621,59671]
TSL = []

for ts in TSA:

    #If the timestamp lies somewhere in March this will use the March file!
    if Time(59639.00000000, format='mjd') <= Time(ts, format='mjd') <= Time(59670.00000000, format='mjd'):
        print("This time lies in March!")
        LocationFile = '../../Data/Location/LIGHT-1_LLA_March.csv'
        LLATData = pd.read_csv(LocationFile)
    
    #If the timestamp lies somewhere in April this will use the April file!
    elif Time(59670.00000000, format='mjd') <= Time(ts, format='mjd') <= Time(59700.00000000, format='mjd'):
        print("This time lies in April!")
        LocationFile = '../../Data/Location/LIGHT-1_LLA_April.csv'
        LLATData = pd.read_csv(LocationFile)
    
    #If the timestamp reached does not exist in the range useable, this will skip that specific value only.
    else:
        print("The time value given does not exist in the range from March to April!\nFor reference, the time in mjd should be between 59639.0 and 59700.00000000")
        continue #Skips the values not in the range!

    #Start the loop that looks through the file.
    for row in range(len(TData)):

        #If the timestamp exists in the data file, use the location already given, and leave the loop. (Condition 1)
        if ts == TData[row]:
            MaximumData = LLATData.T[row]
            MinimumData = LLATData.T[row]
            break
        
        #If the timestamp does not exist in the file, then take the values before and after it, then leave the loop. (Condition 2)
        if TData[row] >= ts:
            MaximumData = LLATData.T[row]
            MinimumData = LLATData.T[row-1]
            break
    
    #If Condition 1 is met, this part will not affect any values.
    #Otherwise, if Condition 2 is met, this part will interpolate the values linearly. 
    #(Hopefully using different interpolation methods later!)
    TimeRange = [MinimumData[0], MaximumData[0]]
    LongRange = [MinimumData[2], MaximumData[2]]
    LatRange  = [MinimumData[1], MaximumData[1]]
    longitude = np.interp(ts, TimeRange, LongRange)
    latitude = np.interp(ts, TimeRange, LatRange)

    #Append the values into an array.
    TSL.append([ts,longitude,latitude])

print(TSL)

The time value given does not exist in the range from March to April!
For reference, the time in mjd should be between 59639.0 and 59700.00000000
This time lies in April!
[[59671, 46.938, -48.851]]


## <span style="color:#72A98F">*Ignore what's below this point*</span>

In [ ]:
# time = E.get_mjd()

# def f(EV:rp.event):
#     time = EV.get_mjd()

In [ ]:
# def g(a):
#     return a+2